In [ ]:
# Import python libraries
#
import os
import importlib
import torch 
import time

# Imports own modules.
#
from loadforecasting_framework import visualization, model_trainer, utils, simulation_config, \
    data_preprocessor
from loadforecasting_framework.simulation_config import *
import loadforecasting_framework
import loadforecasting_models
from loadforecasting_models import Normalizer


In [ ]:
importlib.reload(loadforecasting_models)
importlib.reload(loadforecasting_framework)

# Run the whole simulation
#
train_all_models = False
if train_all_models:
    configs = simulation_config.configs
    model_trainer.ModelTrainer().run(configs)

# Print the sumarized simulation results
#
_ = utils.Evaluate_Models.print_results('outputs/all_train_histories.pkl', 
    'test_loss_relative')


In [ ]:
# Plot the losses over the training.
# (To see, if the models are trained stable.)
#
importlib.reload(loadforecasting_models)
importlib.reload(loadforecasting_framework)

plot_only_single_config = True
plotted_config = \
    ConfigOfOneRun(Model.SIZE_5k, DoTransferLearning.YES, NrOfCommunities.TWENTY, UsedModels.ALL,
        AggregationCount.FIFTY_HOUSEHOLDS, DataSplit.TRAIN_12_MONTH, Epochs.DEFAULT)

utils.Evaluate_Models.plot_training_losses_over_epochs('outputs/all_train_histories.pkl',
                                                       plot_only_single_config,
                                                       plotted_config
                                                       )


In [ ]:
# Evaluate specific model of given power profiles and given configurations
#

importlib.reload(loadforecasting_models)
importlib.reload(loadforecasting_framework)
importlib.reload(utils)

# Define a specific configuration
#
model_type = 'Transformer'
community_id = 0  # chose one of many energy communites
myConfig = \
        ConfigOfOneRun(Model.SIZE_5k, DoTransferLearning.YES, NrOfCommunities.TWENTY, UsedModels.ALL,
        AggregationCount.FIFTY_HOUSEHOLDS, DataSplit.TRAIN_12_MONTH, Epochs.DEFAULT)

modelTrainer = model_trainer.ModelTrainer()
loadProfiles, weatherData, public_holidays_timestamps = modelTrainer.load_data(myConfig)
normalizer = Normalizer()
data_preprocessor = data_preprocessor.DataPreprocessor(
    data_split = myConfig.data_split,
    normalizer = normalizer,
    )

X, Y = data_preprocessor.transform_data(loadProfiles[community_id], weatherData, public_holidays_timestamps)
path_to_trained_parameters = 'outputs/all_trained_models.pth'
num_of_features = X['train'].shape[2]
P_el_predicted = normalizer.de_normalize_y(Y['test']).flatten().unsqueeze(0)
test_profile = os.path.join(os.getcwd(), f"outputs/file_{community_id}.pkl")
my_Model = utils.Deserialize.get_trained_model(path_to_trained_parameters, model_type, test_profile, 
                                                myConfig, num_of_features, data_preprocessor)

# Plot the chosen model
#
plotlyApp = visualization.PlotlyApp(X, Y, data_preprocessor, normalizer, my_Model, None, 'UTC')
plotlyApp.run(myport=8053)


In [ ]:
# Print the model parameter sizes
#

importlib.reload(loadforecasting_models)
importlib.reload(loadforecasting_framework)

model_types = ['xLstm', 'Lstm', 'Transformer']
for model_type in model_types:
    for model_size in ['0.1k', '0.2k', '0.5k', '1k', '2k', '5k', '10k', '20k', '40k', '80k']:

        # Test, if the model is run-able
        num_of_features = 20
        sim_config = ConfigOfOneRun(Model.SIZE_5k, DoTransferLearning.YES, NrOfCommunities.TWENTY, 
            UsedModels.ALL, AggregationCount.FIFTY_HOUSEHOLDS, DataSplit.TRAIN_12_MONTH,
            Epochs.DEFAULT)
        my_model, _ = model_trainer.ModelTrainer.create_model(
            model_type,
            normalizer,
            num_of_features,
            sim_config,
            )

        x = torch.zeros((7, 24, num_of_features))
        my_model(x)

        # Print the model's parameter count
        nr_of_parameters = my_model.get_nr_of_parameters(do_print=False)
        print(f"{model_type} - {model_size} has {nr_of_parameters} parameters.")


In [ ]:
# Print the time needed for training
#

# Vary the model sizes
configs = [ 
    ConfigOfOneRun(Model.SIZE_0k1, DoTransferLearning.YES, NrOfCommunities.TWENTY, UsedModels.ALL,
        AggregationCount.FIFTY_HOUSEHOLDS, DataSplit.TRAIN_12_MONTH, Epochs.DEFAULT),
    ConfigOfOneRun(Model.SIZE_0k2, DoTransferLearning.YES, NrOfCommunities.TWENTY, UsedModels.ALL,
        AggregationCount.FIFTY_HOUSEHOLDS, DataSplit.TRAIN_12_MONTH, Epochs.DEFAULT),
    ConfigOfOneRun(Model.SIZE_0k5, DoTransferLearning.YES, NrOfCommunities.TWENTY, UsedModels.ALL,
        AggregationCount.FIFTY_HOUSEHOLDS, DataSplit.TRAIN_12_MONTH, Epochs.DEFAULT),
    ConfigOfOneRun(Model.SIZE_1k, DoTransferLearning.YES, NrOfCommunities.TWENTY, UsedModels.ALL,
        AggregationCount.FIFTY_HOUSEHOLDS, DataSplit.TRAIN_12_MONTH, Epochs.DEFAULT),
    ConfigOfOneRun(Model.SIZE_2k, DoTransferLearning.YES, NrOfCommunities.TWENTY, UsedModels.ALL,
        AggregationCount.FIFTY_HOUSEHOLDS, DataSplit.TRAIN_12_MONTH, Epochs.DEFAULT),
    ConfigOfOneRun(Model.SIZE_10k, DoTransferLearning.YES, NrOfCommunities.TWENTY, UsedModels.ALL,
        AggregationCount.FIFTY_HOUSEHOLDS, DataSplit.TRAIN_12_MONTH, Epochs.DEFAULT),
    ConfigOfOneRun(Model.SIZE_20k, DoTransferLearning.YES, NrOfCommunities.TWENTY, UsedModels.ALL,
        AggregationCount.FIFTY_HOUSEHOLDS, DataSplit.TRAIN_12_MONTH, Epochs.DEFAULT),
    ConfigOfOneRun(Model.SIZE_40k, DoTransferLearning.YES, NrOfCommunities.TWENTY, UsedModels.ALL,
        AggregationCount.FIFTY_HOUSEHOLDS, DataSplit.TRAIN_12_MONTH, Epochs.DEFAULT),
    ConfigOfOneRun(Model.SIZE_80k, DoTransferLearning.YES, NrOfCommunities.TWENTY, UsedModels.ALL,
        AggregationCount.FIFTY_HOUSEHOLDS, DataSplit.TRAIN_12_MONTH, Epochs.DEFAULT),
    ]

# Loop through the configs and measure the time taken for each
times = []
for config in configs:
    start_time = time.time()  # Record the start time
    model_trainer.ModelTrainer().run([config])
    end_time = time.time()  # Record the end time
    
    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    times.append((config, elapsed_time))

# Print the times for all configs
print("Times for each configuration:")
i = 0
for config, elapsed_time in times:
    i = i + 1
    print(f"Config {i} - Time taken: {elapsed_time/60:.2f} minutes")
